## Train MRCNN Model from pipeline

### Setup and input parms

In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os, sys, math, io, time, gc, platform, pprint
import numpy as np
import tensorflow as tf
import keras
import keras.backend as KB
sys.path.append('../')

import mrcnn.model_mrcnn  as mrcnn_modellib
import mrcnn.model_fcn    as fcn_modellib
import mrcnn.visualize    as visualize

from datetime           import datetime   
from mrcnn.config       import Config
from mrcnn.dataset      import Dataset 
from mrcnn.utils        import log, stack_tensors, stack_tensors_3d, write_stdout
from mrcnn.datagen      import data_generator, load_image_gt
from mrcnn.coco         import CocoDataset, CocoConfig, CocoInferenceConfig, evaluate_coco, build_coco_results
from mrcnn.prep_notebook import mrcnn_coco_train 
from mrcnn.utils        import command_line_parser, Paths
                                
pp = pprint.PrettyPrinter(indent=2, width=100)
np.set_printoptions(linewidth=100,precision=4,threshold=1000, suppress = True)
os_platform = platform.system()

start_time = datetime.now().strftime("%m-%d-%Y @ %H:%M:%S")
print()
print('--> Execution started at:', start_time)
print("    Tensorflow Version: {}   Keras Version : {} ".format(tf.__version__,keras.__version__))

##-----------------------------------------------

### Setup argparse parameters

####  Pass input parameters to argparse

from mrcnn.utils import command_line_parser
parser = command_line_parser()
 # args = parser.parse_args("--epochs 100 --steps_in_epoch 128  --last_epoch 1264 --batch_size 8  --lr 0.5               --logs_dir train_fcn_adagrad --model /home/kbardool/models/train_mrcnn/shapes20180621T1554/mask_rcnn_shapes_1119.h5 --fcn_model init".split())
# input_parms = "--epochs 100 --steps_in_epoch 100  --last_epoch 1264 --batch_size 25 --lr 0.8 --val_steps 5 --logs_dir train_fcn_adagrad --model /home/kbardool/models/train_mrcnn/shapes20180621T1554/mask_rcnn_shapes_1119.h5 --fcn_model /home/kbardool/models/train_fcn_adagrad/shapes20180709T1732/fcn_shapes_1167.h5"
# input_parms +=" --model     /home/kbardool/models/train_mrcnn/shapes20180621T1554/mask_rcnn_shapes_1119.h5 "

input_parms = " --epochs 1  --steps_in_epoch 10  --last_epoch 0 --batch_size 2 --lr 0.001 --val_steps 50 " 
input_parms +=" --mrcnn_logs_dir train_mrcnn_coco " 
input_parms +=" --fcn_logs_dir   train_fcn_coco "
input_parms +=" --mrcnn_model last"
# input_parms +=" --fcn_model /home/kbardool/models/train_fcn_adagrad/shapes20180709T1732/fcn_shapes_1167.h5"
print(input_parms)

args = parser.parse_args(input_parms.split())

##----------------------------------------------------------------------------------------------
## if debug is true set stdout destination to stringIO
##----------------------------------------------------------------------------------------------            
# debug = False
# if debug:
#     sys.stdout = io.StringIO()

print()
print('--> Execution started at:', start_time)
print("    Tensorflow Version: {}   Keras Version : {} ".format(tf.__version__,keras.__version__))

# print("Dataset: ", args.dataset)
# print("Logs:    ", args.logs)
# print("Limit:   ", args.limit)
print("    MRCNN Model        : ", args.mrcnn_model)
print("    FCN Model          : ", args.fcn_model)
print("    MRCNN Log/Ckpt Dir : ", args.mrcnn_logs_dir)
print("    FCN Log/Ckpt  Dir  : ", args.fcn_logs_dir)
print("    Last Epoch         : ", args.last_epoch)
print("    Epochs to run      : ", args.epochs)
print("    Steps in each epoch: ", args.steps_in_epoch)
print("    Validation steps   : ", args.val_steps)
print("    Batch Size         : ", args.batch_size)
print("    Optimizer          : ", type(args.opt), args.opt)
print("    New Log Folder     : ", type(args.new_log_folder), args.new_log_folder)
print("    Sysout             : ", type(args.sysout), args.sysout)
print("    OS Platform        : ", os_platform)



###  Setup Project Directories

##------------------------------------------------------------------------------------
## setup project directories
##   ROOT_DIR         : Root directory of the project 
##   MODEL_DIR        : Directory to save logs and trained model
##   COCO_MODEL_PATH  : Path to COCO trained weights
##---------------------------------------------------------------------------------
paths = Paths(fcn_training_folder = args.fcn_logs_dir, mrcnn_training_folder = args.mrcnn_logs_dir)
paths.display()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload

--> Execution started at: 12-16-2018 @ 18:28:36
    Tensorflow Version: 1.8.0   Keras Version : 2.2.0 
 --epochs 1  --steps_in_epoch 10  --last_epoch 0 --batch_size 2 --lr 0.001 --val_steps 50  --mrcnn_logs_dir train_mrcnn_coco  --fcn_logs_dir   train_fcn_coco  --mrcnn_model last

--> Execution started at: 12-16-2018 @ 18:28:36
    Tensorflow Version: 1.8.0   Keras Version : 2.2.0 
    MRCNN Model        :  last
    FCN Model          :  last
    MRCNN Log/Ckpt Dir :  train_mrcnn_coco
    FCN Log/Ckpt  Dir  :  train_fcn_coco
    Last Epoch         :  0
    Epochs to run      :  1
    Steps in each epoch:  10
    Validation steps   :  50
    Batch Size         :  2
    Optimizer          :  <class 'str'> ADAGRAD
    New Log Folder     :  <class 'bool'> False
    Sysout             :  <class 'str'> SCREEN
    OS Platform        :  Windows
>>> Initialize Paths
 windows  Windows

Paths:
---------------

### Build Configuration Object

In [12]:

##------------------------------------------------------------------------------------
## Build configuration object 
##------------------------------------------------------------------------------------
mrcnn_config                    = CocoConfig()
mrcnn_config.NAME               = 'mrcnn'              
mrcnn_config.DIR_DATASET        = paths.DIR_DATASET
mrcnn_config.DIR_TRAINING       = paths.DIR_TRAINING
mrcnn_config.DIR_PRETRAINED     = paths.DIR_PRETRAINED
mrcnn_config.TRAINING_PATH      = paths.MRCNN_TRAINING_PATH
mrcnn_config.COCO_DATASET_PATH  = paths.COCO_DATASET_PATH 
mrcnn_config.COCO_MODEL_PATH    = paths.COCO_MODEL_PATH   
mrcnn_config.RESNET_MODEL_PATH  = paths.RESNET_MODEL_PATH 
mrcnn_config.VGG16_MODEL_PATH   = paths.VGG16_MODEL_PATH  

mrcnn_config.COCO_CLASSES       = None 
mrcnn_config.DETECTION_PER_CLASS = 200
mrcnn_config.HEATMAP_SCALE_FACTOR = 4



mrcnn_config.BATCH_SIZE         = int(args.batch_size)                  # Batch size is 2 (# GPUs * images/GPU).
mrcnn_config.IMAGES_PER_GPU     = int(args.batch_size)                  # Must match BATCH_SIZE
mrcnn_config.STEPS_PER_EPOCH    = int(args.steps_in_epoch)
mrcnn_config.LEARNING_RATE      = float(args.lr)
mrcnn_config.EPOCHS_TO_RUN      = int(args.epochs)
mrcnn_config.FCN_INPUT_SHAPE    = mrcnn_config.IMAGE_SHAPE[0:2]
mrcnn_config.LAST_EPOCH_RAN     = int(args.last_epoch)
mrcnn_config.WEIGHT_DECAY       = 2.0e-4
mrcnn_config.VALIDATION_STEPS   = int(args.val_steps)
mrcnn_config.REDUCE_LR_FACTOR   = 0.5
mrcnn_config.REDUCE_LR_COOLDOWN = 30
mrcnn_config.REDUCE_LR_PATIENCE = 40
mrcnn_config.EARLY_STOP_PATIENCE= 80
mrcnn_config.EARLY_STOP_MIN_DELTA = 1.0e-4
mrcnn_config.MIN_LR             = 1.0e-10
mrcnn_config.OPTIMIZER            = args.opt
mrcnn_config.NEW_LOG_FOLDER     = args.new_log_folder
mrcnn_config.VERBOSE            = 1  

mrcnn_config.display() 

# print(' Checkpoint directory  : ', mrcnn_config.TRAINING_DIR)
# print(' Checkpoint folder     : ', mrcnn_config.COCO_TRAINING_PATH)
print(' COCO   Dataset Path   : ', mrcnn_config.COCO_DATASET_PATH)
print(' COCO   Model Path     : ', mrcnn_config.COCO_MODEL_PATH)
print(' ResNet Model Path     : ', mrcnn_config.RESNET_MODEL_PATH)
print(' VGG16  Model Path     : ', mrcnn_config.VGG16_MODEL_PATH)
    


Configuration Parameters:
-------------------------
BACKBONE_SHAPES                [[256 256]
 [128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COCO_CLASSES                   None
COCO_DATASET_PATH              F:\MLDatasets\coco2014
COCO_MODEL_PATH                F:\PretrainedModels\mask_rcnn_coco.h5
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
DETECTION_PER_CLASS            200
DIR_DATASET                    F:\MLDatasets
DIR_PRETRAINED                 F:\PretrainedModels
DIR_TRAINING                   F:\models
EARLY_STOP_MIN_DELTA           0.0001
EARLY_STOP_PATIENCE            80
EPOCHS_TO_RUN                  1
FCN_INPUT_SHAPE                [1024 1024]
GPU_COUNT                      1
HEATMAP_SCALE_FACTOR           4
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_MIN

###  Build Model

In [13]:
# mrcnn_model, dataset_train, test_generator, dataset_val, val_generator, mrcnn_config = mrcnn_coco_train(mode = 'training', mrcnn_config = mrcnn_config)
mrcnn_model, mrcnn_config = mrcnn_coco_train(mode = 'training', mrcnn_config = mrcnn_config)


>>> Initialize ModelBase model 
   Mode      :  training
   Model dir :  F:\models\train_mrcnn_coco
>>> ModelBase initialiation complete
>>> ---Initialize MRCNN model, mode:  training
>>> set_log_dir(): model_path:  None
    set_log_dir(): model_path has NOT been provided : None 
                  NewFolder: False  config.NEW_LOG_FOLDER: False 
    set_log_dir(): weight file template (self.checkpoint_path): F:\models\train_mrcnn_coco\mrcnn20181216T1835\mrcnn_{epoch:04d}.h5 
    set_log_dir(): weight file dir      (self.log_dir)        : F:\models\train_mrcnn_coco\mrcnn20181216T1835 
    set_log_dir(): Last completed epoch (self.epoch)          : 0 

----------------------------
>>> Resnet Graph 
----------------------------
     Input_image shape : (?, 1024, 1024, 3)
    After ZeroPadding2D            :  shape: (?, 1030, 1030, 3)    KB.shape:(None, 1030, 1030, 3)  Keras Tensor: True
    After Conv2D padding :         :  shape: (?, 512, 512, 64)     KB.shape:(None, 512, 512, 64)  Keras 

    >> input to MVN.PROB: pos_grid (meshgrid) shape:  (256, 256, ?, 2)
     Prob_grid shape from mvn.probe:  (256, 256, ?)
     Prob_grid shape after tanspose:  (?, 256, 256)
    << output probabilities shape  :  (?, 256, 256)
    old_style_scores        : (2, 81, 200, 3) (2, 81, 200, 3)
    prob_grid_clipped :  (?, 256, 256)

    normalization ------------------------------------------------------
    normalizer     :  (?, 1, 1)
    prob_grid_cns: clipped/normed/scaled :  (?, 256, 256)
    alt_scores_1    :  (None, 3)  Keras tensor  False
    alt_scores_1(by class)       :  (2, 81, 200, 3)  Keras tensor  False

       Normalize_scores() ------------------------------------------------------
         input shape      :  (2, 81, 200, 3)
         reduce_min shape :  (2, 81, 1, 3)
         reduce_max shape :  (2, 81, 1, 3)
             output shape :  (2, 81, 200, 3)

    alt_scores_1_norm(by_class)  :  (2, 81, 200, 3) (2, 81, 200, 3)

    Scatter out the probability distributions based o

    loss      : (?, 200) Tensor("mrcnn_class_loss/Shape_3:0", shape=(2,), dtype=int32) KerasTensor:  False
    mean loss : () Tensor("mrcnn_class_loss/Shape_4:0", shape=(0,), dtype=int32) KerasTensor:  False
    reshaped mean loss : (1, 1) Tensor("mrcnn_class_loss/Shape_5:0", shape=(2,), dtype=int32) KerasTensor:  False

>>> mrcnn_bbox_loss_graph 
    target_class_ids  size : (2, ?)
    pred_bbox size         : (?, 200, 81, 4)
    target_bbox size       : (2, ?, ?)
    reshpaed pred_bbox size         : (?, 81, 4)
    reshaped target_bbox size       : (?, 4)
    pred_bbox size         : (?, 4)
    target_bbox size       : (?, 4)
    loss      : <unknown> Tensor("mrcnn_bbox_loss/Shape:0", shape=(?,), dtype=int32) KerasTensor:  False
    mean loss : () Tensor("mrcnn_bbox_loss/Shape_1:0", shape=(0,), dtype=int32) KerasTensor:  False
    reshaped mean loss : (1, 1) Tensor("mrcnn_bbox_loss/Shape_2:0", shape=(2,), dtype=int32) KerasTensor:  False

>>> mrcnn_bbox_loss_graph 
    target_class_i

In [14]:
# del dataset_train, dataset_val, train_generator, val_generator
from mrcnn.prep_notebook import prep_coco_dataset
dataset_train, train_generator = prep_coco_dataset(['train' , 'val35k'], mrcnn_config, generator=True)
dataset_val  , val_generator   = prep_coco_dataset(['minival'],mrcnn_config, generator=True)

COCO loading annotations file F:\MLDatasets\coco2014\annotations/instances_train2014.json  into memory...
Done (t=20.75s)
creating index...
index created!
 loading  all classes
 image dir            :  F:\MLDatasets\coco2014\train2014
 json_path_dir        :  F:\MLDatasets\coco2014\annotations/instances_train2014.json
 number of images     :  82783
 image_ids[:10]       :  [262145, 131074, 131075, 393221, 393223, 393224, 524297, 393227, 393228, 262146]
 image_ids[1000:1010] :  [22098, 1518, 132591, 1522, 394739, 525813, 1526, 525815, 327935, 394748]
COCO loading annotations file F:\MLDatasets\coco2014\annotations/instances_valminusminival2014.json  into memory...
Done (t=10.54s)
creating index...
index created!
 loading  all classes
 image dir            :  F:\MLDatasets\coco2014\val2014
 json_path_dir        :  F:\MLDatasets\coco2014\annotations/instances_valminusminival2014.json
 number of images     :  35504
 image_ids[:10]       :  [262148, 185686, 360073, 393225, 488823, 240301, 1

In [15]:
print(type(dataset_train), type(train_generator))
print(len(dataset_train.image_info))
print(type(dataset_val), type(val_generator))
print(len(dataset_val.image_info))
print(len(dataset_train.image_ids), len(dataset_val.image_info))
# dataset_train.display_active_class_info()
print(dataset_train.class_from_source_map)
print(dataset_train.class_info)
 

<class 'mrcnn.coco.CocoDataset'> <class 'generator'>
118287
<class 'mrcnn.coco.CocoDataset'> <class 'generator'>
5000
118287 5000
{'coco.57': 52, 'coco.77': 68, 'coco.5': 5, '.0': 0, 'coco.79': 70, 'coco.70': 62, 'coco.61': 56, 'coco.87': 77, 'coco.1': 1, 'coco.51': 46, 'coco.41': 37, 'coco.63': 58, 'coco.82': 73, 'coco.15': 14, 'coco.50': 45, 'coco.10': 10, 'coco.65': 60, 'coco.73': 64, 'coco.36': 32, 'coco.42': 38, 'coco.84': 74, 'coco.33': 29, 'coco.24': 23, 'coco.53': 48, 'coco.44': 40, 'coco.31': 27, 'coco.40': 36, 'coco.13': 12, 'coco.62': 57, 'coco.54': 49, 'coco.7': 7, 'coco.22': 21, 'coco.47': 42, 'coco.37': 33, 'coco.59': 54, 'coco.56': 51, 'coco.23': 22, 'coco.88': 78, 'coco.55': 50, 'coco.9': 9, 'coco.38': 34, 'coco.6': 6, 'coco.64': 59, 'coco.8': 8, 'coco.76': 67, 'coco.85': 75, 'coco.2': 2, 'coco.81': 72, 'coco.34': 30, 'coco.78': 69, 'coco.21': 20, 'coco.39': 35, 'coco.86': 76, 'coco.49': 44, 'coco.17': 16, 'coco.52': 47, 'coco.80': 71, 'coco.11': 11, 'coco.20': 19, 'coc

### Load saved weights

In [12]:
##------------------------------------------------------------------------------------
## Load Mask RCNN Model Weight file
##------------------------------------------------------------------------------------
# exclude_list = ["mrcnn_class_logits"]
#load_model(model, init_with = args.model)   
exclude_list = []
mrcnn_model.load_model_weights(init_with = args.model, exclude = exclude_list)   

-----------------------------------------------
 Load Model with init parm: [ last ]
 Exclude layers: 
-----------------------------------------------
 ---> last
>>> find_last checkpoint in :  /home/kbardool/models/train_mrcnn_coco
 Key : > mrcnn <
 Dir names:  ['mrcnn20181020T1444', 'mrcnn20181013T1146', 'mrcnn20181014T1304', 'mrcnn20181019T1508', 'mrcnn20181012T0928', 'mrcnn20181015T1022', 'mrcnn20181011T1825', 'mrcnn20181011T1117']
 Folder:  /home/kbardool/models/train_mrcnn_coco/mrcnn20181020T1444  Checkpoints:  []
 Folder:  /home/kbardool/models/train_mrcnn_coco/mrcnn20181019T1508  Checkpoints:  []
 Folder:  /home/kbardool/models/train_mrcnn_coco/mrcnn20181015T1022  Checkpoints:  ['mrcnn_0103.h5']
    find_last():   dir_name: /home/kbardool/models/train_mrcnn_coco/mrcnn20181015T1022
    find_last(): checkpoint: /home/kbardool/models/train_mrcnn_coco/mrcnn20181015T1022/mrcnn_0103.h5
   Last file is : /home/kbardool/models/train_mrcnn_coco/mrcnn20181015T1022/mrcnn_0103.h5
>>> find_l

                                3 (512,)  
                               len weight_values: 4   len symbolic_weights: 4
                               i: 0  hdf5 weight shape: (512,)   symb wgt shp: (512,)
                                0 bn3b_branch2c/gamma:0                hdf5 Weights: (512,)  Symbolic Wghts: (512,)
                               i: 1  hdf5 weight shape: (512,)   symb wgt shp: (512,)
                                1 bn3b_branch2c/beta:0                 hdf5 Weights: (512,)  Symbolic Wghts: (512,)
                               i: 2  hdf5 weight shape: (512,)   symb wgt shp: (512,)
                                2 bn3b_branch2c/moving_mean:0          hdf5 Weights: (512,)  Symbolic Wghts: (512,)
                               i: 3  hdf5 weight shape: (512,)   symb wgt shp: (512,)
                                3 bn3b_branch2c/moving_variance:0      hdf5 Weights: (512,)  Symbolic Wghts: (512,)

 58 add_5                     Model Layer Name/Type : [('add_5', <kera

                               Symbolic Weights from Model
                                0 (256,)  
                                1 (256,)  
                                2 (256,)  
                                3 (256,)  
                               len weight_values: 4   len symbolic_weights: 4
                               i: 0  hdf5 weight shape: (256,)   symb wgt shp: (256,)
                                0 bn4d_branch2a/gamma:0                hdf5 Weights: (256,)  Symbolic Wghts: (256,)
                               i: 1  hdf5 weight shape: (256,)   symb wgt shp: (256,)
                                1 bn4d_branch2a/beta:0                 hdf5 Weights: (256,)  Symbolic Wghts: (256,)
                               i: 2  hdf5 weight shape: (256,)   symb wgt shp: (256,)
                                2 bn4d_branch2a/moving_mean:0          hdf5 Weights: (256,)  Symbolic Wghts: (256,)
                               i: 3  hdf5 weight shape: (256,)   symb wgt shp: (256,)

                              Weights     : ['bn4l_branch2c/gamma:0', 'bn4l_branch2c/beta:0', 'bn4l_branch2c/moving_mean:0', 'bn4l_branch2c/moving_variance:0']
                               Symbolic Weights from Model
                                0 (1024,)  
                                1 (1024,)  
                                2 (1024,)  
                                3 (1024,)  
                               len weight_values: 4   len symbolic_weights: 4
                               i: 0  hdf5 weight shape: (1024,)   symb wgt shp: (1024,)
                                0 bn4l_branch2c/gamma:0                hdf5 Weights: (1024,)  Symbolic Wghts: (1024,)
                               i: 1  hdf5 weight shape: (1024,)   symb wgt shp: (1024,)
                                1 bn4l_branch2c/beta:0                 hdf5 Weights: (1024,)  Symbolic Wghts: (1024,)
                               i: 2  hdf5 weight shape: (1024,)   symb wgt shp: (1024,)
                           

                                2 bn4r_branch2b/moving_mean:0          hdf5 Weights: (256,)  Symbolic Wghts: (256,)
                               i: 3  hdf5 weight shape: (256,)   symb wgt shp: (256,)
                                3 bn4r_branch2b/moving_variance:0      hdf5 Weights: (256,)  Symbolic Wghts: (256,)

257 activation_51             Model Layer Name/Type : [('activation_51', <keras.layers.core.Activation object at 0x7f8f289208d0>)] 
                              Weights     : []
                               Symbolic Weights from Model
                               len weight_values: 0   len symbolic_weights: 0

258 res4r_branch2c            Model Layer Name/Type : [('res4r_branch2c', <keras.layers.convolutional.Conv2D object at 0x7f8f2894d208>)] 
                              Weights     : ['res4r_branch2c/kernel:0', 'res4r_branch2c/bias:0']
                               Symbolic Weights from Model
                                0 (1, 1, 256, 1024)  
                

                                0 bn5c_branch2a/gamma:0                hdf5 Weights: (512,)  Symbolic Wghts: (512,)
                               i: 1  hdf5 weight shape: (512,)   symb wgt shp: (512,)
                                1 bn5c_branch2a/beta:0                 hdf5 Weights: (512,)  Symbolic Wghts: (512,)
                               i: 2  hdf5 weight shape: (512,)   symb wgt shp: (512,)
                                2 bn5c_branch2a/moving_mean:0          hdf5 Weights: (512,)  Symbolic Wghts: (512,)
                               i: 3  hdf5 weight shape: (512,)   symb wgt shp: (512,)
                                3 bn5c_branch2a/moving_variance:0      hdf5 Weights: (512,)  Symbolic Wghts: (512,)

336 activation_66             Model Layer Name/Type : [('activation_66', <keras.layers.core.Activation object at 0x7f8f282cdfd0>)] 
                              Weights     : []
                               Symbolic Weights from Model
                               len weig

   Weights file loaded: /home/kbardool/models/train_mrcnn_coco/mrcnn20181015T1022/mrcnn_0103.h5 
>>> Set_log_dir() -- model dir is  /home/kbardool/models/train_mrcnn_coco
  set_log_dir(): self.Checkpoint_path: /home/kbardool/models/train_mrcnn_coco/mrcnn20181020T1500/mrcnn_{epoch:04d}.h5 
  set_log_dir(): self.log_dir        : /home/kbardool/models/train_mrcnn_coco/mrcnn20181020T1500 
  set_log_dir(): Last completed epoch (self.epoch): 0 
MRCNN  MODEL Load weight file COMPLETE    


In [13]:
mrcnn_config.LAST_EPOCH_RAN = 110
mrcnn_config.display()  
mrcnn_model.layer_info()
print()
print("Testing Dataset Image Count: {}".format(len(dataset_train.image_ids)))
print("Testing Dataset Class Count: {}".format(dataset_train.num_classes))


Configuration Parameters:
-------------------------
BACKBONE_SHAPES                [[256 256]
 [128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COCO_CLASSES                   None
COCO_DATASET_PATH              /home/kbardool/MLDatasets/coco2014
COCO_MODEL_PATH                /home/kbardool/PretrainedModels/mask_rcnn_coco.h5
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
DETECTION_PER_CLASS            200
EARLY_STOP_MIN_DELTA           0.0001
EARLY_STOP_PATIENCE            80
EPOCHS_TO_RUN                  1
FCN_INPUT_SHAPE                [1024 1024]
GPU_COUNT                      1
HEATMAP_SCALE_FACTOR           4
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_MIN_DIM                  800
IMAGE_PADDING                  True
IMAGE_SHAPE                    [1024 1024    3]
LAS

### Train mrcnn model : `train()`

In [14]:
mrcnn_model.config.EPOCHS_TO_RUN = 1
mrcnn_model.config.STEPS_PER_EPOCH = 2
mrcnn_model.config.SYSOUT = 'screen'
print('    last epoch ran : ',mrcnn_model.config.LAST_EPOCH_RAN)
print('    epochs to run  : ',mrcnn_model.config.EPOCHS_TO_RUN)
print('    steps per epoch: ',mrcnn_model.config.STEPS_PER_EPOCH)
print('    learning rate  : ', mrcnn_model.config.LEARNING_RATE)
print('    momentum       : ', mrcnn_model.config.LEARNING_MOMENTUM)
print('    weight decay   : ',mrcnn_model.config.WEIGHT_DECAY)


    last epoch ran :  110
    epochs to run  :  1
    steps per epoch:  2
    learning rate  :  0.001
    momentum       :  0.9
    weight decay   :  0.0002


In [15]:
##----------------------------------------------------------------------------------------------
## Train the head branches
## Passing layers="heads" freezes all layers except the head layers. 
## You can also pass a regular expression to select which layers to train by name pattern.
##----------------------------------------------------------------------------------------------

train_layers = [ 'mrcnn', 'fpn','rpn']   ## equivalent to "heads"
loss_names   = [ "rpn_class_loss", "rpn_bbox_loss" , "mrcnn_class_loss", "mrcnn_bbox_loss"]
# train_layers = [ 'mrcnn']
# loss_names   = [ "mrcnn_class_loss", "mrcnn_bbox_loss"]

mrcnn_model.epoch = mrcnn_model.config.LAST_EPOCH_RAN
# mrcnn_model.config.LEARNING_RATE   = 0.001
# mrcnn_model.config.STEPS_PER_EPOCH = config.STEPS_PER_EPOCH
mrcnn_model.train(dataset_train, 
            dataset_val, 
            learning_rate = mrcnn_model.config.LEARNING_RATE, 
            epochs_to_run = mrcnn_model.config.EPOCHS_TO_RUN,
            layers = train_layers,
            losses = loss_names)                  
#             epochs = 25,            # total number of epochs to run (accross multiple trainings)
#             batch_size = 0
#             steps_per_epoch = 0 

            


['mrcnn', 'fpn', 'rpn']
['(mrcnn\\_.*)', '(fpn\\_.*)', '(rpn\\_.*)']
layers regex : (mrcnn\_.*)|(fpn\_.*)|(rpn\_.*)
type train_dataset: <class 'mrcnn.coco.CocoDataset'>
type val_dataset: <class 'mrcnn.coco.CocoDataset'>
    learning rate :  0.001
    momentum      :  0.9


Selecting layers to train
-------------------------
Layer    Layer Name               Layer Type
   0  input_image            (InputLayer          )   ............................no weights to train ]
   1  zero_padding2d_1       (ZeroPadding2D       )   ............................no weights to train ]
   2  conv1                  (Conv2D              )   ............................not a layer we want to train ]
   3  bn_conv1               (BatchNorm           )   ............................not a layer we want to train ]
   4  activation_1           (Activation          )   ............................no weights to train ]
   5  max_pooling2d_1        (MaxPooling2D        )   ............................no weight


L2 Regularization losses:
-------------------------
   Tensor("truediv:0", shape=(), dtype=float32)    name: truediv:0
   Tensor("truediv_1:0", shape=(), dtype=float32)    name: truediv_1:0
   Tensor("truediv_2:0", shape=(), dtype=float32)    name: truediv_2:0
   Tensor("truediv_3:0", shape=(), dtype=float32)    name: truediv_3:0
   Tensor("truediv_4:0", shape=(), dtype=float32)    name: truediv_4:0
   Tensor("truediv_5:0", shape=(), dtype=float32)    name: truediv_5:0
   Tensor("truediv_6:0", shape=(), dtype=float32)    name: truediv_6:0
   Tensor("truediv_7:0", shape=(), dtype=float32)    name: truediv_7:0
   Tensor("truediv_8:0", shape=(), dtype=float32)    name: truediv_8:0
   Tensor("truediv_9:0", shape=(), dtype=float32)    name: truediv_9:0
   Tensor("truediv_10:0", shape=(), dtype=float32)    name: truediv_10:0
   Tensor("truediv_11:0", shape=(), dtype=float32)    name: truediv_11:0
   Tensor("truediv_12:0", shape=(), dtype=float32)    name: truediv_12:0
   Tensor("truediv_13:

/home/kbardool/anaconda3/envs/TFG/lib/python3.5/site-packages/scipy/ndimage/interpolation.py:616: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)
/home/kbardool/anaconda3/envs/TFG/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


-----------------------------------------------------
fit_generator() out_labels:
out_labels:
callback_metrics: ['loss', 'rpn_class_loss', 'rpn_bbox_loss', 'mrcnn_class_loss', 'mrcnn_bbox_loss', 'val_loss', 'val_rpn_class_loss', 'val_rpn_bbox_loss', 'val_mrcnn_class_loss', 'val_mrcnn_bbox_loss']
stateful_metrics: []
-----------------------------------------------------
Epoch 111/111


KeyboardInterrupt: 

### Save 

In [ ]:
# Save weights
# Typically not needed because callbacks save after every epoch
# Uncomment to save manually
model_path = os.path.join(MODEL_DIR, "mask_rcnn_shapes_2500.h5")
model.keras_model.save_weights(model_path)